In [33]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
oppScrData = pd.read_excel(r'sample_data/OppScrData.xlsx')

In [112]:
def clean_ct_data(oppScrData):
    # Delete rows with empty values
    cols = ["L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]']
    ct_data= oppScrData[cols]
    n = ct_data.shape[0]
    preprocessed_ct_data = pd.DataFrame( columns=cols)
    for i in range(n):
        row = ct_data.loc[i]
        ignore = False
        for k,j in enumerate(row):
          if pd.isna(j) or j == ' ': 
            if k == 11:              
              ignore = False
            else:
              ignore = True
              break
        if not ignore:
          preprocessed_ct_data.loc[i]= row
    return preprocessed_ct_data

clinical_data = clean_ct_data(oppScrData=oppScrData)

**Model training**

In [120]:
sc = StandardScaler()

non_null_death_rows = clinical_data[~clinical_data['DEATH [d from CT]'].isnull()]

# Convert to years
non_null_death_rows['DEATH [d from CT]'] /= 365

X = non_null_death_rows.iloc[:, 0:11]
y = non_null_death_rows.iloc[:, 11]
X = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

null_death_rows=clinical_data[clinical_data['DEATH [d from CT]'].isnull()]
X2 = null_death_rows.iloc[:, 0:11]
X2 = pd.DataFrame(sc.fit_transform(X2), columns = X2.columns)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [121]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=0)

In [122]:
RFparams2={'n_estimators':[50],
          'min_samples_split':[16],
          'min_samples_leaf': [4],
           'max_features': [6],
           'max_depth': [20],
           'bootstrap': [True]}

rf = RandomForestRegressor(random_state = 21)
clf = GridSearchCV(rf, RFparams2, refit=True)
clf.fit(X_train, y_train)
clf.best_estimator_.score(X_test, y_test)

-0.0032878970518182005

In [123]:
p = clf.predict(X_test)
Mse = mean_squared_error(y_test, p)
error = mean_absolute_error(y_test, p)
mape = mean_absolute_percentage_error(y_test, p)
print("RMSE : %.2f" % (Mse**(1/2.0)))
print("MApe : %.2f" %(mape))
print("MAE :", error)

RMSE : 3.31
MApe : 4.06
MAE : 2.7602421790301745


**Fill death column**

In [127]:
null_death_rows['DEATH [d from CT]'] = clf.predict(X2)
df = pd.concat([non_null_death_rows,null_death_rows])
y = df.iloc[:,11]
X = df.iloc[:, 0:11]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


**Train model**

In [129]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

knn = KNeighborsRegressor(n_neighbors=9)
param_grid2 = dict(weights = ["uniform", "distance"])
    
grid = GridSearchCV(knn, param_grid2)
grid.fit(X_train, y_train)
p3 = grid.predict(X_test) 
Mse = mean_squared_error(y_test, p3)

mape=mean_absolute_percentage_error(y_test, p3)
print("RMSE: %.2f" % (Mse**(1/2.0)))
print("MApe: %.2f" %(mape))
print("MAE: %.2f" %(mean_absolute_error(y_test, p3)))


RMSE: 1.10
MApe: 0.33
MAE: 0.67
